# Task Distribution
| Member | ID | Tasks |
|---|---|---|
|Bhavika| - | - | 
|Alora| - | - |
|Andrea| - | - |

# Import Libraries

In [37]:
%pip install tweepy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [38]:
import tweepy
import sys
import csv
import pandas as pd
import re
import numpy as np


# Scraping Tweets

In [39]:
# Keys for authentication
consumer_key = "ONLQ0vLSAXM2tZDnckominEcK"
consumer_secret = "MpP7XurdGn1wpufX3rrfQgrs4AROnaw9ZiiZzKN6exr3zEDlN6"
access_token = "1274112117738737664-J2DjjoqqzN11CNp8bpnYzVp1dUYKM4"
access_token_secret = "3coYFJVCqHnOmD2m9Qt6zs7j1I24lhlPTU71eUOq4Zqg9"
bearer_token = "AAAAAAAAAAAAAAAAAAAAALJslQEAAAAA7EWdkTwrVDM44LIuewNascHjvoY%3DeXmhQyuXgo8zXJiSTMfqtmPA393HHO2W7nSrk0bK6b67SuJeb0"


In [40]:
# Authenticate
auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, access_token, access_token_secret)

api = tweepy.API(auth)


In [41]:
def scrape_tweets(query: str, amount: int) -> list:
    """Scrapes a specified number of tweets based on given query and location.

    Method from https://www.sahilfruitwala.com/guide-to-extract-tweets-using-tweepy#how-to-retrieve-specific-number-of-tweets-using-tweepy 

    Args:
        query (str): The query
    Returns:
        list (str): List of contents of tweets.
    """
    extracted_tweets = []
    for tweet in tweepy.Cursor(api.search_tweets, query, count=100, tweet_mode="extended", result_type="recent").items(amount):
        extracted_tweets.append(tweet.full_text)
    return extracted_tweets


In [42]:
def write_to_csv(tweets: list, csv_name: str) -> None:
    """Appends the list of tweets to csv.

    Largely copied from https://gist.github.com/anku255/0cebd75cce675f2b56de1ef48ec06575.

    """
    print("Writing {0} tweets to {1}".format(len(tweets), csv_name))
    tweets_for_csv = [[tweet.encode("utf-8")] for tweet in tweets]
    with open(csv_name, "a+") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerows(tweets_for_csv)


The queries for scraping tweets.

In [43]:

queries = [
    ["(metaverse OR meta verse OR #metaverse OR #meta #verse) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(innovation OR #innovation OR innovate OR innovative OR #innovate OR #innovative) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(sustainability OR #sustainability OR sustainable) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(technology footprint OR technology OR #technologyfootprint) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(artificial intelligence OR ai OR #ai OR #artificialintelligence) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(ethics OR #ethics OR ethical OR #ethical) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(inflation OR #inflation) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(cop 28 OR cop28 OR #cop28 OR #cop #28) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(museum of the future OR @museumofthefuture OR #museumofthefuture OR #museum #of #the #future) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(bit coin OR bitcoin OR #bitcoin OR #crypto OR crpyto OR cryptocurrency) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(cyber attack OR #cyberattack OR #cyber OR #cyberattacks OR #cyber #attack) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(robot OR robots OR #robots OR #robot) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(drone OR drones OR #drone OR #drones) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(hacker OR hacking OR #hacker OR #hacking OR #hack) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(chatgpt OR #chatgpt OR #chat #gpt OR chat gpt) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(cybersecurity OR #cybersecurityOR #cybersec OR cybersec OR cyber security OR #cyber #security) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(sustainabletech OR sustainable ai OR sustainable technology OR #sustainabletech OR #sustainableai OR #sustainabletechnology) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(computing OR quantum computing OR #quantum #computing OR #computing) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(automation OR #automation) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(distributed cloud OR #distributedcloud OR #distributed #cloud) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(big data OR #big #data OR bigdata OR #bigdata) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(ar OR #ar OR #augmentedreality OR #augmented #reality) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(data mining OR #data #mining OR #datamining) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(software OR #software) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(tech OR technology OR #tech OR #technology) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(digital OR digital transformation OR #digital #transformation OR #digitaltransformation) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(blockchain OR #blockchain) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(coders_hq OR coders hq OR #coders OR #coders #hq OR @coders_hq) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(nft OR non-fungible token OR non fungible token OR nfts OR #nfts OR #nonfungibletoken OR #nft) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(binance OR @binance OR #binance) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(uae hackathon OR hackathon OR #hackathon) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(internet of things OR IoT OR #iot OR #internet #of #things OR #internetofthings) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(software engineering OR #softwareengineering) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(#coding OR coding) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(@uaeai OR #uaeai Or #uae #ai) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(technology OR tech OT #tech Or #technology) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(nft OR #nft OR #nfts OR nfts) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(UAE codes OR #UAE_codes OR #UAEcodes) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(Microsoft Hololens OR Hololens OR #Hololens) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(GITEX or #gitex) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(information OR #information) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(internet OR #internet) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(computer OR science OR computer science OR #computerscience OR #computer OR #science) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(technological OR #technological) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(engineering OR #engineering) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(computing OR #computing) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(systems OR #systems) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(electronics OR #electronics) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(application OR #application OR app OR #app) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(robotics OR #robotics) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(high tech OR high technology OR #hightech OR #hightechnology OR #high #technology OR #high #tech) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(nanotech OR nano tech OR nanotechnology OR nano technology OR #nanotech OR #nano #tech OR #nano #technology OR #nanotechnology) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(biotech OR bio tech OR biotechnology OR bio technology OR #biotech OR #bio #tech OR #bio #technology OR #biotechnology) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(code OR #code) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(automation OR #automation OR automate OR #automate) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(virtual OR #virtual OR online OR #online) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(technological advancements OR #technological #advancements OR #advancements OR #techadvancements) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(bionics OR #bionics) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(cyberscience OR #cyberscience) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(google OR #google OR @google) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(oracle OR #oracle OR @oracle) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(ibm OR #ibm OR @ibm) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(bionics OR #bionics) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(sap OR #sap) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(global knowledge OR #globalknowledge OR #global #knowledge) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(amazon OR #amazon OR @amazon) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(hewlett-packard OR Hewlett Packard OR HP OR #hewlett-packard OR #Hewlett #Packard OR #HP) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(cisco OR #cisco OR @cisco) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(dell OR #dell OR @dell) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(cisco OR #cisco OR @cisco) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(@WorldGovSummit OR #WGS OR #WGS2023 OR World Goverment Summit) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(masdar OR #masdar OR @masdar) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(renewable energy OR clean energy OR #renewableenergy OR #cleanenergy) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(machine learning OR #machinelearning OR ML OR #ML) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(recommender systems OR #RS OR #recommender #systems OR recommender engine) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(deep learning OR #deeplearning) AND (place_country:AE OR uae) lang:en -filter:retweets"],
    ["(research OR #research) AND (place_country:AE OR uae) lang:en -filter:retweets"],
]


Commented out below to prevent re-running code.

In [44]:
# """Scraping tweets and writing them to dataset.csv"""
# csv_name="dataset.csv"
# total = 0
# for query in queries:
#     tweets = scrape_tweets(query=query, amount=500)
#     total += len(tweets)
#     print("Found {0} tweets related to the query {1}".format(len(tweets), query))
#     write_to_csv(tweets=tweets,csv_name=csv_name)
# print("Done. {0} rows.".format(total))


# Data Cleaning

This section cleans the scraped tweets dataset of duplicate tweets, URLs, @'s and #s.

Note: 
- Do not modify original csv `dataset.csv`. The dataset with no duplicates is stored in `updated_dataset.csv`

In [45]:
# Dropping duplicates
dataset = pd.read_csv("dataset.csv", header=None)
dataset.rename(columns={0: 'Text'}, inplace=True)


In [46]:
dataset


,Text
0,b'Joined UAE Minister &amp; President-Designat...
1,b'Foreign Secretary @AmbVMKwatra participated ...
2,"b'As the UAE marks National Environment Day, w..."
3,b'RT @KhalifaAlgaz: Did you know that the UAE ...
4,b'RT @tawhidChtioui: \xf0\x9f\x9b\xb0\xef\xb8\...
...,...
21974,b'Send our special gifts to your loved ones!\x...
21975,b'Trainer code: 2735 9457 4031\nfrom \xf0\x9f\...
21976,"b""We are looking for ICY SNOW , SANDSTORM .\nI..."
21977,b'Make #MONEY from #home through PC #APP \nMak...


In [47]:
updated_dataset = dataset.drop_duplicates().copy()
updated_dataset


,Text
0,b'Joined UAE Minister &amp; President-Designat...
1,b'Foreign Secretary @AmbVMKwatra participated ...
2,"b'As the UAE marks National Environment Day, w..."
3,b'RT @KhalifaAlgaz: Did you know that the UAE ...
4,b'RT @tawhidChtioui: \xf0\x9f\x9b\xb0\xef\xb8\...
...,...
21888,b'@m_ut67 @modgovae @Forsan_UAE Dear sir pleas...
21889,b'@UAE_BARQ Dear sir please my help you sir my...
21890,b'@UAE_BARQ Dear sir please my help you sir my...
21915,"b""#PhonePe launched a service on Tuesday that ..."


### Removing URLs, @'s and hashtags

In [48]:
tweets = updated_dataset['Text'].copy()
updated_dataset['Cleaned Text'] = tweets.str.replace(
    r'@[^\s]+|#[^\s]+|https?:\/\/\S+|www\.\S+', '', regex=True)
updated_dataset['Cleaned Text'].head()


0    b'Joined UAE Minister &amp; President-Designat...
1    b'Foreign Secretary  participated in a meeting...
2    b'As the UAE marks National Environment Day, w...
3    b'RT  Did you know that the UAE is developing ...
4    b'RT  \xf0\x9f\x9b\xb0\xef\xb8\x8f UAE lunar r...
Name: Cleaned Text, dtype: object

### Removing Duplicates again

In [49]:
updated_dataset = updated_dataset.drop('Text', axis=1)
updated_dataset = updated_dataset.drop_duplicates()
updated_dataset.shape


(5398, 1)

In [50]:
updated_dataset.to_csv('updated-dataset.csv')


In [51]:
updated_dataset = pd.read_csv('updated-dataset.csv', index_col=0)
updated_dataset


,Cleaned Text
0,b'Joined UAE Minister &amp; President-Designat...
1,b'Foreign Secretary participated in a meeting...
2,"b'As the UAE marks National Environment Day, w..."
3,b'RT Did you know that the UAE is developing ...
4,b'RT \xf0\x9f\x9b\xb0\xef\xb8\x8f UAE lunar r...
...,...
21888,b' Dear sir please my help you sir my two ki...
21889,b' Dear sir please my help you sir my two kidn...
21890,b' Dear sir please my help you sir my two kidn...
21915,"b"" launched a service on Tuesday that will all..."


# Data Labelling


### TextBlob

For data labelling, we decided to use first use TextBlob. TextBlob calculates the subjectivity and polarity of a text to classify the text as 'Positive', 'Negative' or 'Neutral'

In [52]:
# %pip install TextBlob


In [53]:
from textblob import TextBlob


In [54]:
# Function to get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity


# Function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity


# Create two new columns ‘Subjectivity’ & ‘Polarity’
updated_dataset['TextBlob Subjectivity'] = updated_dataset['Cleaned Text'].apply(
    getSubjectivity)
updated_dataset['TextBlob Polarity'] = updated_dataset['Cleaned Text'].apply(
    getPolarity)


def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'


updated_dataset['TextBlob Sentiment'] = updated_dataset['TextBlob Polarity'].apply(
    getAnalysis)


In [55]:
updated_dataset['TextBlob Sentiment'].value_counts()


Positive    2643
Neutral     2276
Negative     479
Name: TextBlob Sentiment, dtype: int64

It can be observed that TextBlob classified the majority of tweets to be Positive. To compare, we also decided to use Vader as well to observe if there was a major difference in classification.

### Vader

In [56]:
# %pip install vaderSentiment


In [57]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sentiment = SentimentIntensityAnalyzer()


def getSentiment(text):
    return sentiment.polarity_scores(text)


updated_dataset['Vader Analysis'] = updated_dataset['Cleaned Text'].apply(
    getSentiment)


def sentimentAnalysis(sentiment_dict):
    if sentiment_dict['compound'] >= 0.05:
        return 'Positive'
    elif sentiment_dict['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'


updated_dataset['Vader Sentiment'] = updated_dataset['Vader Analysis'].apply(
    sentimentAnalysis)


In [58]:
updated_dataset[['Vader Analysis', 'Vader Sentiment']]


,Vader Analysis,Vader Sentiment
0,"{'neg': 0.0, 'neu': 0.638, 'pos': 0.362, 'comp...",Positive
1,"{'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'comp...",Positive
2,"{'neg': 0.0, 'neu': 0.766, 'pos': 0.234, 'comp...",Positive
3,"{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'comp...",Positive
4,"{'neg': 0.0, 'neu': 0.819, 'pos': 0.181, 'comp...",Positive
...,...,...
21888,"{'neg': 0.073, 'neu': 0.726, 'pos': 0.202, 'co...",Positive
21889,"{'neg': 0.073, 'neu': 0.726, 'pos': 0.202, 'co...",Positive
21890,"{'neg': 0.074, 'neu': 0.719, 'pos': 0.206, 'co...",Positive
21915,"{'neg': 0.043, 'neu': 0.774, 'pos': 0.183, 'co...",Positive


In [59]:
updated_dataset['Vader Sentiment'].value_counts()


Positive    3335
Neutral     1691
Negative     372
Name: Vader Sentiment, dtype: int64

## Analysis between Vader and TextBlob


| Data Labelling Method      | Positive Label | Negative Label | Neutral Label |
| ----------- | ----------- | ----------- | ----------- |
| TextBlob      | 2643   (49%)   |479  (8.9%)| 2276 (42.1%)|
| Vader   | 3335  (61.8%)      | 372 (6.9%) | 1691 (31.3%)|

From the summary table, we can observe that the negatively labelled tweets stay within a 2% difference of each other, regardless of the data labelling method. However, the positive and neutral label differ by 10%.

In [60]:
# Creating a csv of tweets with conflicting sentiments
unclearTweets = updated_dataset.loc[(
    updated_dataset['Vader Sentiment'] != updated_dataset['TextBlob Sentiment'])]
unclearTweets.to_csv('ConflictingLabels.csv')


In [61]:
unclearTweets


,Cleaned Text,TextBlob Subjectivity,TextBlob Polarity,TextBlob Sentiment,Vader Analysis,Vader Sentiment
1,b'Foreign Secretary participated in a meeting...,0.562500,-0.062500,Negative,"{'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'comp...",Positive
3,b'RT Did you know that the UAE is developing ...,0.000000,0.000000,Neutral,"{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'comp...",Positive
4,b'RT \xf0\x9f\x9b\xb0\xef\xb8\x8f UAE lunar r...,1.000000,-0.600000,Negative,"{'neg': 0.0, 'neu': 0.819, 'pos': 0.181, 'comp...",Positive
10,b'RT UK BUSINESS SECRETARY PRAISES UK-UAE GRA...,0.000000,0.000000,Neutral,"{'neg': 0.0, 'neu': 0.484, 'pos': 0.516, 'comp...",Positive
14,"b'RT As host of COP28, UAE is committed to le...",0.125000,0.000000,Neutral,"{'neg': 0.0, 'neu': 0.817, 'pos': 0.183, 'comp...",Positive
...,...,...,...,...,...,...
21888,b' Dear sir please my help you sir my two ki...,0.100000,0.000000,Neutral,"{'neg': 0.073, 'neu': 0.726, 'pos': 0.202, 'co...",Positive
21889,b' Dear sir please my help you sir my two kidn...,0.100000,0.000000,Neutral,"{'neg': 0.073, 'neu': 0.726, 'pos': 0.202, 'co...",Positive
21890,b' Dear sir please my help you sir my two kidn...,0.100000,0.000000,Neutral,"{'neg': 0.074, 'neu': 0.719, 'pos': 0.206, 'co...",Positive
21915,"b"" launched a service on Tuesday that will all...",0.062500,-0.062500,Negative,"{'neg': 0.043, 'neu': 0.774, 'pos': 0.183, 'co...",Positive


After manually reviewing the differences in labels, it seems that the TextBlob sentiment labelling is more accurate.

# Data Preprocessing

**approach → first data cleaning + tokenization + lemmatization**

## Additional Data Cleaning

### Removing Non-English Characters

In [62]:
def nonenglish(tweet):
    # tweets in different language changed to hex code \xhh
    tweet = re.sub(r'\\x[a-zA-Z0-9]+', '', tweet)
    tweet = re.sub(r'\\n', '', tweet)  # removing new line character as well
    return tweet


updated_dataset['Cleaned Text'] = updated_dataset['Cleaned Text'].apply(
    lambda tweet: nonenglish(tweet))
updated_dataset['Cleaned Text'].head()


0    b'Joined UAE Minister &amp; President-Designat...
1    b'Foreign Secretary  participated in a meeting...
2    b'As the UAE marks National Environment Day, w...
3    b'RT  Did you know that the UAE is developing ...
4    b'RT   UAE lunar rover will test 1st artificia...
Name: Cleaned Text, dtype: object

### Removing Stopwords

In [63]:
from sklearn.feature_extraction import text

# contains 318 stopwords, including but not limited to articles and prepositions
stopwords = text.ENGLISH_STOP_WORDS

updated_dataset['Cleaned Text'] = updated_dataset['Cleaned Text'].apply(
    lambda tweet: ' '.join([text for text in tweet.split(' ') if text not in stopwords]))
updated_dataset['Cleaned Text'].head()


0    b'Joined UAE Minister &amp; President-Designat...
1    b'Foreign Secretary  participated meeting foca...
2    b'As UAE marks National Environment Day, remai...
3    b'RT  Did know UAE developing Arabic ChatGPT u...
4    b'RT   UAE lunar rover test 1st artificial int...
Name: Cleaned Text, dtype: object

### Removing Symbols and Punctuations

In [64]:
import string

punctuations = string.punctuation
print(punctuations)

updated_dataset['Cleaned Text'] = updated_dataset['Cleaned Text'].apply(
    lambda tweet: tweet.translate({ord(punc): None for punc in punctuations}))
updated_dataset['Cleaned Text'].head()


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


0    bJoined UAE Minister amp PresidentDesignate  H...
1    bForeign Secretary  participated meeting focal...
2    bAs UAE marks National Environment Day remain ...
3    bRT  Did know UAE developing Arabic ChatGPT us...
4    bRT   UAE lunar rover test 1st artificial inte...
Name: Cleaned Text, dtype: object

### Removing Numbers

In [65]:
updated_dataset['Cleaned Text'] = updated_dataset['Cleaned Text'].apply(
    lambda tweet: re.sub('[0-9]+', '', tweet))
updated_dataset['Cleaned Text'].head()


0    bJoined UAE Minister amp PresidentDesignate  H...
1    bForeign Secretary  participated meeting focal...
2    bAs UAE marks National Environment Day remain ...
3    bRT  Did know UAE developing Arabic ChatGPT us...
4    bRT   UAE lunar rover test st artificial intel...
Name: Cleaned Text, dtype: object

### Removing 'b' and 'RT'

In [66]:
updated_dataset['Cleaned Text'] = updated_dataset['Cleaned Text'].apply(
    lambda tweet: re.sub('bRT|b', '', tweet))
updated_dataset['Cleaned Text'].head()


0    Joined UAE Minister amp PresidentDesignate  HE...
1    Foreign Secretary  participated meeting focal ...
2    As UAE marks National Environment Day remain c...
3      Did know UAE developing Araic ChatGPT using ...
4       UAE lunar rover test st artificial intellig...
Name: Cleaned Text, dtype: object

### Tokenization

In [67]:
# as data has been completely cleaned, we can simply tokenize by converting strings to lists
updated_dataset['Tokenized Text'] = updated_dataset['Cleaned Text'].apply(
    lambda tweet: tweet.split())
updated_dataset['Tokenized Text'].head()


0    [Joined, UAE, Minister, amp, PresidentDesignat...
1    [Foreign, Secretary, participated, meeting, fo...
2    [As, UAE, marks, National, Environment, Day, r...
3    [Did, know, UAE, developing, Araic, ChatGPT, u...
4    [UAE, lunar, rover, test, st, artificial, inte...
Name: Tokenized Text, dtype: object

### Lemmatization

In [68]:
import nltk
lemmatizing = nltk.stem.WordNetLemmatizer()

updated_dataset['Lemmatized Text'] = updated_dataset['Tokenized Text'].apply(
    lambda tweet: [lemmatizing.lemmatize(text).lower() for text in tweet])
updated_dataset['Lemmatized Text'].head()


0    [joined, uae, minister, amp, presidentdesignat...
1    [foreign, secretary, participated, meeting, fo...
2    [as, uae, mark, national, environment, day, re...
3    [did, know, uae, developing, araic, chatgpt, u...
4    [uae, lunar, rover, test, st, artificial, inte...
Name: Lemmatized Text, dtype: object

In [69]:
# Encode TextBlob Sentiments into numbers
updated_dataset['Encoded_TextBlob_Sentiment'] = updated_dataset['TextBlob Sentiment'].apply(
    lambda x: 1 if x == 'Positive' else -1 if x == 'Negative' else 0)


In [70]:
# resets the index numbers of the dataframe
updated_dataset.reset_index(inplace=True, drop=True)
updated_dataset


,Cleaned Text,TextBlob Subjectivity,TextBlob Polarity,TextBlob Sentiment,Vader Analysis,Vader Sentiment,Tokenized Text,Lemmatized Text,Encoded_TextBlob_Sentiment
0,Joined UAE Minister amp PresidentDesignate HE...,0.200000,0.133333,Positive,"{'neg': 0.0, 'neu': 0.638, 'pos': 0.362, 'comp...",Positive,"[Joined, UAE, Minister, amp, PresidentDesignat...","[joined, uae, minister, amp, presidentdesignat...",1
1,Foreign Secretary participated meeting focal ...,0.562500,-0.062500,Negative,"{'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'comp...",Positive,"[Foreign, Secretary, participated, meeting, fo...","[foreign, secretary, participated, meeting, fo...",-1
2,As UAE marks National Environment Day remain c...,0.525000,0.166667,Positive,"{'neg': 0.0, 'neu': 0.766, 'pos': 0.234, 'comp...",Positive,"[As, UAE, marks, National, Environment, Day, r...","[as, uae, mark, national, environment, day, re...",1
3,Did know UAE developing Araic ChatGPT using ...,0.000000,0.000000,Neutral,"{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'comp...",Positive,"[Did, know, UAE, developing, Araic, ChatGPT, u...","[did, know, uae, developing, araic, chatgpt, u...",0
4,UAE lunar rover test st artificial intellig...,1.000000,-0.600000,Negative,"{'neg': 0.0, 'neu': 0.819, 'pos': 0.181, 'comp...",Positive,"[UAE, lunar, rover, test, st, artificial, inte...","[uae, lunar, rover, test, st, artificial, inte...",-1
...,...,...,...,...,...,...,...,...,...
5393,Dear sir help sir kidney Damage Doctor disc...,0.100000,0.000000,Neutral,"{'neg': 0.073, 'neu': 0.726, 'pos': 0.202, 'co...",Positive,"[Dear, sir, help, sir, kidney, Damage, Doctor,...","[dear, sir, help, sir, kidney, damage, doctor,...",0
5394,Dear sir help sir kidney Damage Doctor discus...,0.100000,0.000000,Neutral,"{'neg': 0.073, 'neu': 0.726, 'pos': 0.202, 'co...",Positive,"[Dear, sir, help, sir, kidney, Damage, Doctor,...","[dear, sir, help, sir, kidney, damage, doctor,...",0
5395,Dear sir help sir kidney Damage Doctor discus...,0.100000,0.000000,Neutral,"{'neg': 0.074, 'neu': 0.719, 'pos': 0.206, 'co...",Positive,"[Dear, sir, help, sir, kidney, Damage, Doctor,...","[dear, sir, help, sir, kidney, damage, doctor,...",0
5396,launched service Tuesday allow Indian users t...,0.062500,-0.062500,Negative,"{'neg': 0.043, 'neu': 0.774, 'pos': 0.183, 'co...",Positive,"[launched, service, Tuesday, allow, Indian, us...","[launched, service, tuesday, allow, indian, us...",-1


# Text Representation

This section experiments with different document representations, namely:
- Bag of Words
- N-grams
- TF-IDF
- CBOW
- Skip-gram
- Pre-trained Word2Vec model by Google


Each of the representations are tested using Naive Bayes, and the high performance models are used in the next stage of the pipeline.

Note: Arbitrarily using lemmatization and textblob sentiments

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline


Getting train and test sets ready

In [72]:
X = updated_dataset['Lemmatized Text']
y = updated_dataset['Encoded_TextBlob_Sentiment']
X_train, X_test, y_train, y_test = train_test_split(
    updated_dataset['Lemmatized Text'],
    updated_dataset['Encoded_TextBlob_Sentiment'],
    test_size=0.2,
    random_state=42,
    stratify=updated_dataset['Encoded_TextBlob_Sentiment']
)


## Bag Of Words

In [174]:

bow = CountVectorizer(
    ngram_range=(1, 1),  # unigram
    preprocessor=lambda x: x,  # override preprocessing cause already done
    tokenizer=lambda x: x,  # override tokenization cause already done
)

nb_bow = Pipeline([
    ('bow', bow),
    ('naive_bayes', MultinomialNB())
])

nb_bow.fit(X_train, y_train)
y_pred_bow = nb_bow.predict(X_test)
print(classification_report(y_test, y_pred_bow))


              precision    recall  f1-score   support

          -1       0.71      0.28      0.40        96
           0       0.85      0.58      0.69       455
           1       0.67      0.92      0.77       529

    accuracy                           0.72      1080
   macro avg       0.74      0.60      0.62      1080
weighted avg       0.75      0.72      0.71      1080



## N-grams
The n-grams tested are $(a,b)$, which denotes a vector representation where a-grams up until b-grams are used.
E.g. When the n-gram range is (1,3), the vector representation of the document includes unigrams, bigrams, and trigrams. 

If the range is $(a,a)$, only a-grams are used. E.g. (1,1) denotes unigrams.

Possible n-gram values that are tested are:
$\text{n-gram values}=\{(a,b) | a \in \{1,2,...,9\} \cap b \in \{1,2,...,9\} \cap (a <= b)\}$

It is observed that the n-gram range (1,6) provides the best macro-f1 score.

In [106]:
from sklearn.metrics import precision_recall_fscore_support
A = np.arange(1, 10)
B = np.arange(1, 10)

# Generate possible n-gram pairs
ngram_pairs = [(a, b) for a in A for b in B if a <= b]

results_ngram = {}
for ngram_range in ngram_pairs:
    ngram = CountVectorizer(
        ngram_range=ngram_range,
        preprocessor=lambda x: x,  # override preprocessing
        tokenizer=lambda x: x,  # override tokenization
    )

    ngram_nb = Pipeline([
        ('ngram', ngram),
        ('naive_bayes', MultinomialNB())
    ])

    ngram_nb.fit(X_train, y_train)
    ngram_nb_y_pred = ngram_nb.predict(X_test)

    prec, recall, f1, support = precision_recall_fscore_support(
        y_test, ngram_nb_y_pred, beta=1.0, average='macro')
    results_ngram[ngram_range] = {
        'precision': prec, 'recall': recall, 'f1-score': f1}


In [116]:
ngram_results = pd.DataFrame.from_dict(results_ngram, orient='index', columns=[
                                       'precision', 'recall', 'f1-score'])
ngram_results.index.names = ['min_ngram', 'max_ngram']
# get n-grams corresponding to max f1 score
ngram_results.idxmax()


precision    (1, 3)
recall       (1, 6)
f1-score     (1, 6)
dtype: object

In [154]:
ngram_results


precision    recall  f1-score
min_ngram max_ngram                               
1         1           0.744011  0.595388  0.623261
          2           0.776213  0.624163  0.656949
          3           0.778171  0.633833  0.666656
          4           0.768264  0.636147  0.667612
          5           0.760271  0.637612  0.667404
          6           0.761317  0.638243  0.668040
          7           0.761317  0.638243  0.668040
          8           0.761317  0.638243  0.668040
          9           0.761317  0.638243  0.668040
2         2           0.703705  0.616992  0.643597
          3           0.695476  0.616362  0.641273
          4           0.695689  0.616362  0.641320
          5           0.699136  0.616362  0.642112
          6           0.699891  0.616992  0.642742
          7           0.699891  0.616992  0.642742
          8           0.699891  0.616992  0.642742
          9           0.699891  0.616992  0.642742
3         3           0.716251  0.607867  0.634091
          4           0.716762  0.608600  0.634875
          5           0.719255  0.607969  0.635058
          6           0.719255  0.607969  0.635058
          7           0.719255  0.607969  0.635058
          8           0.718739  0.607237  0.634275
          9           0.718739  0.607237  0.634275
4         4           0.718298  0.589320  0.614002
          5           0.721500  0.592792  0.617818
          6           0.721500  0.592792  0.617818
          7           0.721974  0.593525  0.618640
          8           0.721974  0.593525  0.618640
          9           0.721500  0.592792  0.617818
5         5           0.732348  0.574538  0.598222
          6           0.732792  0.575270  0.599076
          7           0.732348  0.574538  0.598222
          8           0.732348  0.574538  0.598222
          9           0.732348  0.574538  0.598222
6         6           0.731875  0.557748  0.578741
          7           0.731875  0.557748  0.578741
          8           0.731875  0.557748  0.578741
          9           0.730415  0.557118  0.578145
7         7           0.720098  0.539068  0.555661
          8           0.718542  0.538438  0.555073
          9           0.718542  0.538438  0.555073
8         8           0.708012  0.506673  0.515449
          9           0.708012  0.506673  0.515449
9         9           0.699506  0.479202  0.478806

## TF-IDF

The TF-IDF vectorizor is tested over several n-gram ranges. The ranges are: $\text{n-gram values}=\{(a,b) | a \in \{1,2,...,9\} \cap b \in \{1,2,...,9\} \cap (a <= b)\}$.

It is observed that the tfidf vector with n-gram range (2,3) provides the best macro-f1 score.

In [123]:
from sklearn.feature_extraction.text import TfidfVectorizer

A = np.arange(1, 10)
B = np.arange(1, 10)

# Generate possible n-gram pairs
ngram_pairs = [(a, b) for a in A for b in B if a <= b]

results_tfidf = {}
for ngram_range in ngram_pairs:
    tfidf = TfidfVectorizer(
        preprocessor=lambda x: x,  # override preprocessing
        tokenizer=lambda x: x,  # override tokenization
        ngram_range=ngram_range
    )

    tfidf_nb = Pipeline([
        ('tfidf', tfidf),
        ('naive_bayes', MultinomialNB())
    ])

    tfidf_nb.fit(X_train, y_train)
    tfidf_nb_y_pred = tfidf_nb.predict(X_test)
    prec, recall, f1, support = precision_recall_fscore_support(
        y_test, tfidf_nb_y_pred, beta=1.0, average='macro')
    results_tfidf[ngram_range] = {
        'precision': prec, 'recall': recall, 'f1-score': f1}


In [124]:
tfidf_results = pd.DataFrame.from_dict(results_tfidf, orient='index', columns=[
                                       'precision', 'recall', 'f1-score'])
tfidf_results.index.names = ['min_ngram', 'max_ngram']
# get n-grams corresponding to max f1 score
tfidf_results.idxmax()


precision    (1, 4)
recall       (1, 4)
f1-score     (2, 3)
dtype: object

In [125]:
tfidf_results


precision    recall  f1-score
min_ngram max_ngram                               
1         1           0.840404  0.534808  0.540647
          2           0.848358  0.544962  0.551621
          3           0.848872  0.551467  0.560676
          4           0.851383  0.560170  0.571980
          5           0.848805  0.558178  0.569967
          6           0.848156  0.558280  0.570094
          7           0.844364  0.556492  0.568337
          8           0.843167  0.556697  0.568583
          9           0.843468  0.558265  0.570198
2         2           0.815706  0.555756  0.572937
          3           0.794665  0.556064  0.573077
          4           0.795223  0.555331  0.572561
          5           0.795223  0.555331  0.572561
          6           0.817078  0.555229  0.572929
          7           0.818145  0.549545  0.562975
          8           0.818489  0.549442  0.562859
          9           0.817971  0.548709  0.562124
3         3           0.815290  0.533663  0.546693
          4           0.795003  0.538923  0.553380
          5           0.792883  0.535451  0.547788
          6           0.791359  0.533150  0.545197
          7           0.820266  0.535773  0.548283
          8           0.819241  0.535143  0.547675
          9           0.818469  0.530938  0.541159
4         4           0.817540  0.523642  0.535187
          5           0.816789  0.521532  0.530587
          6           0.816789  0.521532  0.530587
          7           0.816789  0.521532  0.530587
          8           0.817902  0.522162  0.531187
          9           0.820148  0.523423  0.532389
5         5           0.819089  0.513284  0.520406
          6           0.819089  0.513284  0.520406
          7           0.818834  0.509812  0.514577
          8           0.819616  0.509709  0.514328
          9           0.819363  0.506237  0.508392
6         6           0.817659  0.499336  0.499927
          7           0.817659  0.499336  0.499927
          8           0.817659  0.499336  0.499927
          9           0.817415  0.495864  0.493884
7         7           0.815303  0.477081  0.464636
          8           0.813333  0.477184  0.464853
          9           0.812094  0.473711  0.458244
8         8           0.805167  0.451529  0.430413
          9           0.805167  0.451529  0.430413
9         9           0.795174  0.431002  0.400267

## Word2Vec

In [130]:
from gensim.models import Word2Vec
import gensim

# %pip install gensim


In [131]:
from sklearn.preprocessing import MinMaxScaler


def normalize_vectors(input_vectors):
    """Normalizes vectors to be in range [0,1)

    Args:
        input_vectors (np.ndarray): List of vectors, each vector is np.ndarray
    """

    return MinMaxScaler().fit(input_vectors).transform(input_vectors)


In [137]:
def vectorize_sentence(sentence, wv_model, mode=0):
    """Convert a sentence to a vector by summing word vectors in the sentence.

    Another option could be to average word vectors rather than just sum.

    Arg: 
        sentence (list): Tokenized sentence
        mode (int, 1 or 0): 0 for summing word vectors, 1 or averaging word vectors.
    Returns:
        (np.ndarray): Vector representation of sentence
    """
    if mode not in [0, 1]:
        raise Exception("Mode parameter should be either 1 or 0")

    vector_size = wv_model.vector_size
    wv_res = np.zeros(vector_size)
    count = 1
    for w in sentence:
        if w in wv_model:
            wv_res += wv_model[w]
            if mode:
                count += 1
    if mode:
        wv_res = wv_res/count
    return wv_res


In [133]:
def vectorize_sentence_2(sentences, wv_model, mode=0):
    """Convert a list of sentences to a vectors.

    By summing word vectors in the sentence.
    Another option could be to average word vectors rather than just sum.

    Arg: 
        sentence (pd.Series): Series consisting of list of tokenized texts 
        mode (int, 1 or 0): 0 for summing word vectors, 1 or averaging word vectors.
    Returns:
        (np.ndarray): Vector representation of sentences
    """
    if mode not in [0, 1]:
        raise Exception("Mode parameter should be either 1 or 0")

    vector_size = wv_model.vector_size
    count = 1
    vector_sentences = []

    for i in range(len(sentences)):
        sentence = np.zeros(vector_size)
        for word in sentences[i]:
            if word in wv_model:
                count += 1
                sentence += wv_model[word]
        if mode:  # if mode set to 1 (average word vectors)
            sentence = sentence/count
        vector_sentences.append(sentence)
    vector_sentences_df = pd.DataFrame(vector_sentences)
    return vector_sentences_df


## CBOW
The parameters tested are window, cbow_modes, and vec_mode.
- window determines how many context words around the target word to use.
- cbow_mode is either 1 or 0, and indicates whether the context word vectors are summed(0) or averaged(1).
- vec_mode is either 1 or 0, and indicates whether the document vector should be the sum(0) or average(1) of its word vectors.  


It is observed that the highest macro-f1 score is obtained when the window size is 1, cbow_mode is 1, and the vec_mode is 0.

In [156]:
# parameter values
windows = np.arange(1, 6)
cbow_modes = [1, 0]
vectorization_modes = [0, 1]

# create the cartesian product of possible combinaiton of parameters
parameters = [(window, cbow_mode, vec_mode)
              for window in windows for cbow_mode in cbow_modes for vec_mode in vectorization_modes]

results_cbow = {}
for window, cbow_mode, vec_mode in parameters:
    cbow = Word2Vec(
        sentences=X,
        window=window,  # how many context words before and after target word to consider
        min_count=1,  # ignore words with frequency lower than this
        sg=0,  # 0 to use cbow, 1 for skipgram
        cbow_mean=cbow_mode,  # 0 uses sum of context word vectors, 1 uses mean
    )
    cbow.train(X, total_examples=cbow.corpus_count, epochs=cbow.epochs)
    # After training, KeyedVector object is used
    cbow_model = cbow.wv

    # Get the embeddings of train and test set
    X_train_cbow = normalize_vectors(
        np.array([vectorize_sentence(x, cbow_model, vec_mode) for x in X_train]))
    X_test_cbow = normalize_vectors(
        np.array([vectorize_sentence(x, cbow_model, vec_mode) for x in X_test]))

    # Test NB
    nb_cbow = MultinomialNB()
    nb_cbow.fit(X_train_cbow, y_train)
    y_pred_cbow = nb_cbow.predict(X_test_cbow)

    prec, recall, f1, support = precision_recall_fscore_support(
        y_test, y_pred_cbow, beta=1.0, average='macro', zero_division=0)
    results_cbow[(window, cbow_mode, vec_mode)] = {
        'precision': prec, 'recall': recall, 'f1-score': f1}


In [157]:

cbow_results = pd.DataFrame.from_dict(results_cbow, orient='index', columns=[
                                      'precision', 'recall', 'f1-score'])
cbow_results.index.names = ['window size',
                            'cbow mean value', 'document vectorization mode']
# get n-grams corresponding to max f1 score
cbow_results.idxmax()

precision    (1, 0, 1)
recall       (1, 1, 0)
f1-score     (1, 1, 0)
dtype: object

In [175]:
cbow_results

precision    recall  \
window size cbow mean value document vectorization mode                        
1           1               0                             0.406385  0.431903   
                            1                             0.408865  0.383704   
            0               0                             0.394161  0.425044   
                            1                             0.410892  0.383807   
2           1               0                             0.375491  0.410916   
                            1                             0.399158  0.381696   
            0               0                             0.399660  0.428210   
                            1                             0.403268  0.382854   
3           1               0                             0.373843  0.410460   
                            1                             0.398996  0.380861   
            0               0                             0.386608  0.419050   
                            1                             0.408466  0.384744   
4           1               0                             0.365628  0.401946   
                            1                             0.391754  0.382106   
            0               0                             0.391583  0.422216   
                            1                             0.393045  0.381578   
5           1               0                             0.362904  0.399205   
                            1                             0.385898  0.379381   
            0               0                             0.388525  0.420838   
                            1                             0.397530  0.381066   

                                                         f1-score  
window size cbow mean value document vectorization mode            
1           1               0                            0.410402  
                            1                            0.334447  
            0               0                            0.404993  
                            1                            0.335113  
2           1               0                            0.392110  
                            1                            0.335466  
            0               0                            0.407584  
                            1                            0.335870  
3           1               0                            0.391289  
                            1                            0.333822  
            0               0                            0.399493  
                            1                            0.337342  
4           1               0                            0.382712  
                            1                            0.337584  
            0               0                            0.402164  
                            1                            0.337736  
5           1               0                            0.379704  
                            1                            0.334482  
            0               0                            0.401080  
                            1                            0.334975

## Skip Gram

The parameters tested are window, and vec_mode.
- window determines how many context words around the target word to use.
- vec_mode is either 1 or 0, and indicates whether the document vector should be the sum(0) or average(1) of its word vectors.  


It is observed that the highest macro-f1 score is obtained when the window size is 4 and the vec_mode is 0.

In [142]:
windows = np.arange(1, 6)
vectorization_modes = [0, 1]

skipgram_parameters = [(window, vec_mode)
                       for window in windows for vec_mode in vectorization_modes]

results_sg = {}
for window, vec_mode in skipgram_parameters:
    skipgram = Word2Vec(
        sentences=X,
        window=window,  # how many context words before and after target word to consider
        min_count=1,  # ignore words with frequency lower than this
        sg=1,  # 0 to use cbow, 1 for skipgram
    )
    skipgram.train(X, total_examples=skipgram.corpus_count,
                   epochs=skipgram.epochs)
    # After training, KeyedVector object is used
    skipgram_model = skipgram.wv

    # Get the embeddings of train and test set
    X_train_sg = normalize_vectors(
        np.array([vectorize_sentence(x, skipgram_model, vec_mode) for x in X_train]))
    X_test_sg = normalize_vectors(
        np.array([vectorize_sentence(x, skipgram_model, vec_mode) for x in X_test]))

    # Test NB
    nb_sg = MultinomialNB()
    nb_sg.fit(X_train_sg, y_train)
    y_pred_sg = nb_sg.predict(X_test_sg)

    prec, recall, f1, support = precision_recall_fscore_support(
        y_test, y_pred_sg, beta=1.0, average='macro', zero_division=0)
    results_sg[(window, vec_mode)] = {
        'precision': prec, 'recall': recall, 'f1-score': f1}


modes: 1, 0
modes: 1, 1
modes: 2, 0
modes: 2, 1
modes: 3, 0
modes: 3, 1
modes: 4, 0
modes: 4, 1
modes: 5, 0
modes: 5, 1


In [144]:
skipgram_results = pd.DataFrame.from_dict(results_sg, orient='index', columns=[
                                          'precision', 'recall', 'f1-score'])
skipgram_results.index.names = ['window size', 'document vectorization mode']
# get n-grams corresponding to max f1 score
skipgram_results.idxmax()
# skipgram_results.max()


precision    (2, 1)
recall       (4, 0)
f1-score     (4, 0)
dtype: object

In [176]:
skipgram_results

precision    recall  f1-score
window size document vectorization mode                               
1           0                             0.395672  0.424854  0.404232
            1                             0.420314  0.387485  0.337363
2           0                             0.390294  0.425101  0.405688
            1                             0.757977  0.393155  0.347706
3           0                             0.724105  0.427021  0.411113
            1                             0.750041  0.400612  0.364189
4           0                             0.728356  0.429556  0.413292
            1                             0.757443  0.404597  0.368270
5           0                             0.395744  0.425264  0.405005
            1                             0.756287  0.403235  0.366747

### Using Pre-trained Word2Vec Model
Using pre-trained word2vec model from Google, which has 3 million words. Trained on 100 billion words from the google news dataset. Unsure if it uses CBOW or SkipGram or both

From https://thinkingneuron.com/how-to-classify-text-using-word2vec/
Google Model can be downloaded from https://thinkingneuron.com/how-to-classify-text-using-word2vec/#:~:text=download%20link%3A%20https%3A%2F%2Fdrive.google.com%2Ffile%2Fd%2F0b7xkcwpi5kdynlnuttlss21pqmm%2Fedit%3Fusp%3Dsharing

In [147]:
# load the word vectors from Google Model (1GB)
# Will take time because 3 mil words.
# Download model from https://thinkingneuron.com/how-to-classify-text-using-word2vec/#:~:text=download%20link%3A%20https%3A%2F%2Fdrive.google.com%2Ffile%2Fd%2F0b7xkcwpi5kdynlnuttlss21pqmm%2Fedit%3Fusp%3Dsharing
GoogleModel = gensim.models.KeyedVectors.load_word2vec_format(
    '/mnt/d/Andrea/Heriot-Watt/year-4/f20aa/f20aa-coursework-1/GoogleNews-vectors-negative300.bin', binary=True)


The 2 ways to vectorize documents using the Word2Vec model is either by summing (mode=1) or averaging (mode=1) each of the word vectors in the document.

It is obeserved that the highest macro-f1 score is obtained when the word vectors are summed.

In [170]:
modes = np.arange(0,2)
w2v_data_mode0 = 0
w2v_data_mode1 = 0

results_google_model = {}
for mode in modes:
    print(mode)
    w2v_data = vectorize_sentence_2(X, GoogleModel, mode=mode)
    
    # Get train and test values
    X_w2v = w2v_data.values
    y_w2v = y.values

    # scale because NB doesnt accept negative values
    X_w2v_scaled = normalize_vectors(X_w2v)

    X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(
        X_w2v_scaled, y_w2v, test_size=0.2, random_state=42,
        stratify=y)

    nb = MultinomialNB()
    nb.fit(X_train_w2v, y_train_w2v)
    y_pred_w2v = nb.predict(X_test_w2v)

    prec, recall, f1, support = precision_recall_fscore_support(
        y_test_w2v, y_pred_w2v, beta=1.0, average='macro', zero_division=0)
    results_google_model[mode] = {
        'precision': prec, 'recall': recall, 'f1-score': f1}


0
1


In [172]:
google_model_results = pd.DataFrame.from_dict(
    results_google_model, orient='index', columns=['precision', 'recall', 'f1-score'])
google_model_results.index.names = ['document vectorization mode']
# get n-grams corresponding to max f1 score
google_model_results.idxmax()


precision    0
recall       0
f1-score     0
dtype: int64

In [173]:
google_model_results

,precision,recall,f1-score
document vectorization mode,,,
0,0.437536,0.475932,0.454780
1,0.163114,0.332703,0.218905


## Comparison of Text Representation Models
The different models are compared based on their macroaveraged precision, recall, and f1 score.
Values that are closer to one are optimal.

|Model|Precision|Recall|F-1|
|---|---|---|---|
|Bag Of Words (Unigram)|0.74|0.60|0.62|
|N-gram range (1,6)|0.76|0.64|0.67|
|TF-IDF on n-gram range (2,3)|0.79|0.56|0.57|
|CBOW (Trained on scraped tweets corpus)|0.41|0.43|0.41|
|Skip-gram (Trained on scraped tweets corpus)|0.73|0.43|0.41|
|Word2Vec model (Trained on Google News Corpus)|0.437536|0.48|0.45|


It was expected that the more sophisticated models - those which take into account the context of words - would perform better. However, based on the results the more primitive approaches performed better. This could be due to our small dataset.

The n-gram model with range (1,6) was the best performer in terms of macro F-1 score, which is the unweighted harmonic mean of the precision and recall values.




# Classification Models

### 1. Logistic Regression with N-grams range (1,6)

In [177]:
from sklearn.linear_model import LogisticRegression

ngram = CountVectorizer(
    ngram_range=(1, 6),  # ngrams where n is 1 through 6
    preprocessor=lambda x: x,  # override preprocessing
    tokenizer=lambda x: x,  # override tokenization
)

ngram_lr = Pipeline([
    ('ngram', ngram),
    ('logistic_regression', LogisticRegression())
])

ngram_lr.fit(X_train, y_train)
ngram_lr_y_pred = ngram_lr.predict(X_test)
print(classification_report(y_test, ngram_lr_y_pred))


              precision    recall  f1-score   support

          -1       0.78      0.30      0.44        96
           0       0.74      0.86      0.80       455
           1       0.82      0.79      0.80       529

    accuracy                           0.78      1080
   macro avg       0.78      0.65      0.68      1080
weighted avg       0.78      0.78      0.77      1080



### 2. FastText with Trigrams

FastText is an open-source library created by Facebook AI Research lab. Setup instructions are here: https://fasttext.cc/docs/en/supervised-tutorial.html#installing-fasttext

In [178]:
pip install fasttext


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 346.1 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4133920 sha256=8c4d33402fd131b4474303dfc57f9111c382ffdebb43d9b08384b28533ff1da8
  Stored in directory: /home/andrea/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
Note: you may need to restart the kernel to use updated packages.


In [179]:
import fasttext
help(fasttext.FastText)


Help on module fasttext.FastText in fasttext:

NAME
    fasttext.FastText

DESCRIPTION
    # Copyright (c) 2017-present, Facebook, Inc.
    # All rights reserved.
    #
    # This source code is licensed under the MIT license found in the
    # LICENSE file in the root directory of this source tree.

FUNCTIONS
    cbow(*kargs, **kwargs)
    
    eprint(*args, **kwargs)
    
    load_model(path)
        Load a model given a filepath and return a model object.
    
    read_args(arg_list, arg_dict, arg_names, default_values)
    
    skipgram(*kargs, **kwargs)
    
    supervised(*kargs, **kwargs)
    
    tokenize(text)
        Given a string of text, tokenize it and return a list of tokens
    
    train_supervised(*kargs, **kwargs)
        Train a supervised model and return a model object.
        
        input must be a filepath. The input text does not need to be tokenized
        as per the tokenize function, but it must be preprocessed and encoded
        as UTF-8. You might wan

Reformatting the dataset as FastText expects each row of data to be in the form:
__label__sentiment tweet 

In [180]:
ftDf = updated_dataset[['TextBlob Sentiment', 'Cleaned Text']].copy()
ftDf.iloc[:, 0] = ftDf.iloc[:, 0].apply(lambda x: '__label__' + x)
ftDf.shape


(5398, 2)

In [55]:
# Splitting into test and train
training = ftDf.sample(frac=0.7)
testing = ftDf.drop(training.index)

training.to_csv('train.txt',
                index=False,
                sep=' ',
                header=None,
                quoting=csv.QUOTE_NONE,
                quotechar="",
                escapechar=" ")

testing.to_csv('test.txt',
               index=False,
               sep=' ',
               header=None,
               quoting=csv.QUOTE_NONE,
               quotechar="",
               escapechar=" ")


In [56]:
ftModel = fasttext.train_supervised('train.txt', wordNgrams=3)


In [59]:
ftModel.test('test.txt')


(1619, 0.44224830142063004, 0.44224830142063004)

To evaluate further, we can test it on unseen samples.

In [82]:
ftModel.predict(
    'I love the UAE for its innovation with artificial intelligence.')


(('__label__Neutral',), array([0.46955368]))

On the test set, fastText model had a precision score of 0.44 ad a recall score of 0.44. It had also misclassified an unseen sample. However, FastText model parameters can be adjusted, so we will try adjusting the epoch and learning rate.

In [81]:
ftModel2 = fasttext.train_supervised(
    'train.txt', wordNgrams=3, epoch=6, lr=0.25)
ftModel2.test('test.txt')


(1619, 0.7794935145151328, 0.7794935145151328)

In [83]:
ftModel2.predict(
    'I love the UAE for its innovation with artificial intelligence.')


(('__label__Positive',), array([0.68560129]))

As shown above, increasing the epoch and learning rate corrected a misclassification that happened with the previous model.
